### 1. Importing Packages, DF Set-up, Defining Outdated Tech/Processes

In [1]:
import re
import numpy as np
import pandas as pd
import xml.etree.ElementTree
from lxml import etree
import os
import io

#Hide Future Version warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#show all the columns when dataframes are displayed
pd.set_option('display.max_columns', 500)

In [4]:
#Using the original flat table
df = pd.read_excel("AI_Reg_Flat_Table_V3_Part1.xlsx")

In [5]:
df

,Instrumentation_Num,Registration_Date,Consolidation_Date,Last_Mod_Date,Enabling_Authority,Short_Title,Long_Title,Reg_Maker,Reg_Order_Num,Reg_Order_Maker_Date,Schedule,Order
0,"C.R.C., c. 10",NaN,06/21/2018,05/26/2005,['AERONAUTICS ACT'],Flying Accidents Compensation Regulations,Regulations Prescribing Compensation for Bodil...,NaN,NaN,NaN,NaN,NaN
1,"C.R.C., c. 100",NaN,06/21/2018,6/23/2005,['AERONAUTICS ACT'],Ottawa International Airport Zoning Regulations,Regulations Respecting Zoning at Ottawa Intern...,NaN,NaN,NaN,"[{'OriginatingRef': '(Sections 2 and 4)', 'Lab...",NaN
2,"C.R.C., c. 101",NaN,06/21/2018,2/24/2012,['AERONAUTICS ACT'],Penticton Airport Zoning Regulations,Regulations Respecting Zoning at Penticton Air...,NaN,NaN,NaN,"[{'OriginatingRef': '(Sections 2 and 4)', 'Lab...",NaN
3,"C.R.C., c. 1013",NaN,06/21/2018,7/19/2005,['CANADA LABOUR CODE'],Canada Industrial Relations Remuneration Regul...,Regulations Respecting the Remuneration and Ex...,NaN,NaN,NaN,NaN,NaN
4,"C.R.C., c. 1015",NaN,06/21/2018,7/19/2005,['FAIR WAGES AND HOURS OF LABOUR ACT'],NaN,Fair Wages and Hours of Labour Regulations,NaN,NaN,NaN,NaN,NaN
5,"C.R.C., c. 102",NaN,06/21/2018,2/24/2012,['AERONAUTICS ACT'],Pitt Meadows Airport Zoning Regulations,Regulations Respecting Zoning at Pitt Meadows ...,NaN,NaN,NaN,"[{'OriginatingRef': '(Sections 2 and 4)', 'Lab...",NaN
6,"C.R.C., c. 1021",NaN,06/21/2018,7/19/2005,['CANADA LANDS SURVEYS ACT'],Lands Surveys Tariff,Tariff of Fees to be Charged by the Department...,NaN,NaN,NaN,"[{'OriginatingRef': '(Section 2)', 'Label': 'S...",NaN
7,"C.R.C., c. 1022",NaN,06/21/2018,7/21/2005,['LIEUTENANT GOVERNORS SUPERANNUATION ACT'],Lieutenant Governors Superannuation Regulations,Regulations Made Pursuant to the Lieutenant Go...,NaN,NaN,NaN,NaN,NaN
8,"C.R.C., c. 1023",NaN,06/21/2018,10/17/2007,['HEALTH OF ANIMALS ACT'],Hatchery Regulations,Regulations Respecting the Improvement of Poul...,NaN,NaN,NaN,NaN,NaN
9,"C.R.C., c. 1026",NaN,06/21/2018,7/19/2005,['LIVESTOCK FEED ASSISTANCE ACT'],Canadian Livestock Feed Board Headquarters Reg...,Regulations Designating the Headquarters of th...,NaN,NaN,NaN,NaN,NaN


In [6]:
#Outdated items to search for 
list_outdated_tech = ['paper', 'print', 'mail', 'fax', 'pen', 'pencil', 'ink', 'carbon copy', 'hard copy', 
                      'original copy', 'on site','on-site', 'file', 'filing']

list_outdated_process = ['written', 'write', 'wrote', 'physical', 'proof', 'signed', 'in person', 'in-person', 
                         'notary', 'submission', 'non-electronic', 'non electronic', 
                         'document (excluding electronic)', 'excluding electronic', 'signature']

### 2. Required (Helper) Fucntions 

In [7]:
#Code to convert XML element tree to a dictionary
def make_dict_from_tree(element_tree):
    def internal_iter(tree, accum):
        if tree is None:
            return accum
        
        if tree.getchildren():
            accum[tree.tag] = {}
            for each in tree.getchildren():
                result = internal_iter(each, {})
                if each.tag in accum[tree.tag]:
                    if not isinstance(accum[tree.tag][each.tag], list):
                        accum[tree.tag][each.tag] = [
                            accum[tree.tag][each.tag]
                        ]
                    accum[tree.tag][each.tag].append(result[each.tag])
                else:
                    accum[tree.tag].update(result)
        else:
            accum[tree.tag] = tree.text
        return accum
    return internal_iter(element_tree, {})

In [8]:
#function to clean each body text
def body_clean(text):
    cleaned = text.replace("Text", "")
    cleaned = cleaned.replace("[", "")
    cleaned = cleaned.replace("]", "")
    cleaned = cleaned.replace("{", "")
    cleaned = cleaned.replace("}", "")
    return cleaned

#function to clean each label tag
def label_clean(text):
    cleaned = body_clean(text)
    cleaned = cleaned.replace("'", "")
    return cleaned

In [9]:
#function to help update dictionaries if a key does or does not exist
def set_key(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = value
        
    elif type(dictionary[key]) == list:
        dictionary[key].append(value)
        
    else:
        dictionary[key] = [dictionary[key], value]

### 3. Formatting the SOR Regulations & Updating the DF Body Column

In [10]:
#Obtain the list of regulations only from the working directory
list_regs = []
files = [f for f in os.listdir('.') if os.path.isfile(f)]

for file in files:
    if file[-3:] == 'xml':
        list_regs.append(file)

In [11]:
#Checking the number of regulations
print (len(list_regs))
print (len(df))

3199
3199


In [12]:
#Final list of each regulation's formatted body text to append to the dataframe
list_body_final = []
list_exceptions = []

for reg in list_regs:
    #Create the master dictionary for each regulation 
    doc = etree.parse(reg)
    doc_dict = make_dict_from_tree(doc.getroot())
    
    #Create the nested list for each regulation's label and associated text 
    ###Structure for one regulation --> [[Label: X, [phrase1, phrase2, phrase3]],[Label: Y, [Phrase5, Phrase6]]]
    list_reg_labeled_body = []  
    
    #The following code will execute assuming the reg follows the conventional structure...track the exceptions
    try:
        #Iterate through each part of a regulation's body (i.e. the labels such as label 1, label 2 etc.)
        Parts = len(doc_dict['Regulation']['Body']['Section'])

        for i in range(0,Parts):
            #Identify the title (e.g.number) of each section in the regulation body
            label_number = str(doc_dict['Regulation']['Body']['Section'][i]["Label"])
            label_tag = "Label: " + label_clean(label_number)

            #Determine all the contents associated with the label, removing the label key
            label_text = doc_dict['Regulation']['Body']['Section'][i].copy()
            del label_text['Label']

            #Format the text for each label, removing remains of xml tags and dictionary formatting
            label_text = body_clean(str(label_text))
            label_text = label_text.split("'")
            list_label_text = []

            for phrase in label_text:
                if len(phrase) > 2:
                    list_label_text.append(phrase)

            #Create the nested pair of label_tag + label_text --> [Label: X, [phrase1, phrase2, phrase3]]
            label_pair = []
            label_pair.append(label_tag)
            label_pair.append(list_label_text)

            #Append all the nested lists for each individual regulation
            list_reg_labeled_body.append(label_pair)   

        #Append each regulations final label-text nested list for adding to the df
        list_body_final.append(list_reg_labeled_body)
        
    except:
        list_body_final.append("")
        list_exceptions.append(reg)

In [13]:
#Checking the length of the outputs
print (len(list_body_final))
print (len(list_exceptions))

3199
77


In [14]:
#Updating the main df 'Body' column
df["Body"] = list_body_final

### 4. Counting the Outdated Item Occurances

In [15]:
#List of formatted body text 
list_body = df['Body']

In [16]:
for tech in list_outdated_tech:  
    outdated_tech_count = []
    
    for body in list_body:
        #use the '\\b' word boundary to only return exact matches
        n = re.findall('\\b' + tech + '\\b', str(body).lower())
        outdated_tech_count.append(len(n))
        
    df[tech] = outdated_tech_count
df['Outdated_Tech_Total'] = df[list_outdated_tech].sum(axis=1)

In [17]:
for process in list_outdated_process:
    outdated_process_count = []
    
    for body in list_body:
        #use the '\\b' word boundary to only return exact matches
        n = re.findall('\\b' + process + '\\b', str(body).lower())
        outdated_process_count.append(len(n))
        
    df[process] = outdated_process_count

df['Outdated_Process_Total'] = df[list_outdated_process].sum(axis=1)

In [18]:
#Checking the output - regulations with outdated tech (any)
df[df['Instrumentation_Num']=='SOR/2000-187']

,Instrumentation_Num,Registration_Date,Consolidation_Date,Last_Mod_Date,Enabling_Authority,Short_Title,Long_Title,Reg_Maker,Reg_Order_Num,Reg_Order_Maker_Date,Schedule,Order,Body,paper,print,mail,fax,pen,pencil,ink,carbon copy,hard copy,original copy,on site,on-site,file,filing,Outdated_Tech_Total,written,write,wrote,physical,proof,signed,in person,in-person,notary,submission,non-electronic,non electronic,document (excluding electronic),excluding electronic,signature,Outdated_Process_Total
586,SOR/2000-187,5/4/2000,06/21/2018,12/1/2005,['AGRICULTURE AND AGRI-FOOD ADMINISTRATIVE MON...,NaN,Agriculture and Agri-Food Administrative Monet...,NaN,NaN,NaN,NaN,"<Provision format-ref=""indent-0-0"" language-al...","[[Label: 1, [The following definitions apply i...",0,0,9,8,0,0,0,0,1,0,0,0,0,0,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 5. Extracting the Outdated Phrases & Locations

##### Outdated Tech

In [19]:
#Final lists to be appended to the df --> {'tech':'labelX':['phrase1'],['phrase2']}
outdated_phrases_tech = []

#looking at each regulation individually and checking for all outdated tech one at a time 
for body in list_body:
    
    #Format for each regulation will be a nested dictionary
    ## {tech1:{label1:[phrase1, phrase2], label2:[phrase3]}, tech2:{label6:[phrase5,phrase6]}, etc..}
    master_dict_tech = {}
    
    for tech in list_outdated_tech:
    #Iterate through each regulations body 'label' and the list of text 'phrases' for each 'label'
        for label in range(0,len(body)):
            label_name = str(body[label][0])
            label_phrase_dict = {}
            
            for phrase in body[label][1]:
                #check if the outdated item exists (exact match only)
                if len(re.findall('\\b' + tech + '\\b', str(phrase).lower())) > 0:
                    #Use the pre defined dictionary helper function, checks if key already exists
                    set_key(label_phrase_dict, label_name, phrase)
            
            #Only add to the master dictionary if outdated tech was found (not empty dictionary)
            if bool(label_phrase_dict):
                set_key(master_dict_tech, tech, label_phrase_dict)
        
    #Append the final master dict to the list
    outdated_phrases_tech.append(master_dict_tech)
    
#update the final dataframe 
df['Outdated_Phrases_Tech'] = outdated_phrases_tech

##### Outdated Process

In [20]:
#Final lists to be appended to the df --> {'tech':'labelX':['phrase1'],['phrase2']}
outdated_phrases_process = []

#looking at each regulation individually and checking for all outdated tech one at a time 
for body in list_body:
    
    #Format for each regulation will be a nested dictionary
    ## {tech1:{label1:[phrase1, phrase2], label2:[phrase3]}, tech2:{label6:[phrase5,phrase6]}, etc..}
    master_dict_process = {}

    for process in list_outdated_process:
    #Iterate through each regulations body 'label' and the list of text 'phrases' for each 'label'
        for label in range(0,len(body)):
            label_name = str(body[label][0])
            label_phrase_dict = {}
            
            for phrase in body[label][1]:
                #check if the outdated item exists (exact match only)
                if len(re.findall('\\b' + process + '\\b', str(phrase).lower())) > 0:
                    #Use the pre defined dictionary helper function, checks if key already exists
                    set_key(label_phrase_dict, label_name, phrase)
            
            #Only add to the master dictionary if outdated tech was found (not empty dictionary)
            if bool(label_phrase_dict):
                set_key(master_dict_process, process, label_phrase_dict)
        
    #Append the final master dict to the list
    outdated_phrases_process.append(master_dict_process)
        
#update the final dataframe 
df['Outdated_Phrases_Process'] = outdated_phrases_process

##### Exporting to Excel

In [109]:
df['Outdated_Phrases_Process'][17]

{'written': {'Label: 7': 'Every licensee shall obtain from each nuclear energy worker who is informed of the matters referred to in paragraphs (1)(a) and (b) and subsection (2) a written acknowledgement that the worker has received the information.'}}

In [21]:
#Export the df to xlms 
df.to_excel('AI_Reg_Flat_Table_V3_Final.xlsx')

### 6. Analyzing the Outputs

In [386]:
df['Body'][0]

[['Label: 1',
  ['The definitions in this section apply in these Regulations.',
   'Definition',
   'XRefExternal',
   'Public Service Superannuation Act',
   'DefinedTermFr',
   'Loi',
   'DefinedTermEn',
   'Act',
   'DefinedTermFr',
   'nouvel employeur',
   'DefinedTermEn',
   'new employer']],
 ['Label: 2',
  ['Subsection',
   'XRefExternal',
   'Divestiture of Service Transitional Coverage Regulations',
   'Label',
   '(1)',
   'These Regulations do not apply to a person who subsequently becomes re-employed by the new employer.',
   'Label',
   '(2)',
   'Sections 4 to 9 do not apply to a person who has exercised an option under subsection 3(2).',
   'Label',
   '(3)',
   'HistoricalNote',
   'SOR/2016-203, s. 74(E).']],
 ['Label: 3',
  ['Subsection',
   'Sections 12, 13 and 13.01 of the Act only apply to a person on and after the date on which that person ceases to be employed by the new employer.',
   'Label',
   '(1)',
   'Despite subsection (1), where, on or after January 1, 

In [384]:
df['Outdated_Phrases_Tech'][15].keys()

['mail', 'fax', 'hard copy']

In [407]:
for tech in list_outdated_tech:
    print (tech + " : " + str(len(df[df[tech]>0])))

paper : 136
print : 99
mail : 316
fax : 80
pen : 731
pencil : 5
ink : 65
carbon copy : 0
hard copy : 4
original copy : 5
proof : 94
on site : 38
file : 238
filing : 100
submit : 360
submission(non-electronic) : 0
non-electronic : 2
document (excluding electronic) : 0
excluding electronic : 0


In [409]:
for process in list_outdated_process:
    print (process + " : " + str(len(df[df[process]>0])))

written : 380
write : 28
wrote : 0
physical : 129
proof : 94
signed : 417
in person : 70
in-person : 0
original copy : 5
notary : 16
submit : 360
submission(non-electronic) : 0
non-electronic : 2
document (excluding electronic) : 0
excluding electronic : 0
written : 380
signature : 69


In [387]:
list_regs

['SOR-2000-1.xml',
 'SOR-2000-100.xml',
 'SOR-2000-107.xml',
 'SOR-2000-108.xml',
 'SOR-2000-111.xml',
 'SOR-2000-112.xml',
 'SOR-2000-113.xml',
 'SOR-2000-131.xml',
 'SOR-2000-132.xml',
 'SOR-2000-14.xml',
 'SOR-2000-141.xml',
 'SOR-2000-142.xml',
 'SOR-2000-143.xml',
 'SOR-2000-177.xml',
 'SOR-2000-181.xml',
 'SOR-2000-187.xml',
 'SOR-2000-202.xml',
 'SOR-2000-203.xml',
 'SOR-2000-204.xml',
 'SOR-2000-205.xml',
 'SOR-2000-206.xml',
 'SOR-2000-207.xml',
 'SOR-2000-208.xml',
 'SOR-2000-209.xml',
 'SOR-2000-210.xml',
 'SOR-2000-211.xml',
 'SOR-2000-212.xml',
 'SOR-2000-214.xml',
 'SOR-2000-217.xml',
 'SOR-2000-228.xml',
 'SOR-2000-230.xml',
 'SOR-2000-233.xml',
 'SOR-2000-253.xml',
 'SOR-2000-260.xml',
 'SOR-2000-265.xml',
 'SOR-2000-272.xml',
 'SOR-2000-273.xml',
 'SOR-2000-277.xml',
 'SOR-2000-283.xml',
 'SOR-2000-294.xml',
 'SOR-2000-300.xml',
 'SOR-2000-303.xml',
 'SOR-2000-306.xml',
 'SOR-2000-324.xml',
 'SOR-2000-375.xml',
 'SOR-2000-376.xml',
 'SOR-2000-387.xml',
 'SOR-2000-418.x